In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [7]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化到[-1, 1]
])

In [9]:
# 加载CIFAR-10数据集
batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [11]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 卷积层
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 输入通道3，输出通道32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 输入通道32，输出通道64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 输入通道64，输出通道128
        
        # 全连接层
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # 输入大小128*4*4，输出大小256
        self.fc2 = nn.Linear(256, 128)  # 输入大小256，输出大小128
        self.fc3 = nn.Linear(128, 10)  # 输入大小128，输出大小10（10个类别）
        
        # 池化层和Dropout
        self.pool = nn.MaxPool2d(2, 2)  # 2x2最大池化
        self.dropout = nn.Dropout(0.25)  # Dropout概率0.25

    def forward(self, x):
        # 卷积层1 + ReLU + 池化
        x = self.pool(F.relu(self.conv1(x)))
        # 卷积层2 + ReLU + 池化
        x = self.pool(F.relu(self.conv2(x)))
        # 卷积层3 + ReLU + 池化
        x = self.pool(F.relu(self.conv3(x)))
        
        # 展平
        x = x.view(-1, 128 * 4 * 4)
        
        # 全连接层1 + ReLU + Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # 全连接层2 + ReLU + Dropout
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        # 全连接层3（输出层）
        x = self.fc3(x)
        
        return x

In [13]:

# 实例化模型
model = SimpleCNN()

In [15]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器

In [17]:
# 训练模型
num_epochs = 10  # 训练10个epoch
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        # 清零梯度
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        # 打印统计信息
        running_loss += loss.item()
        if i % 200 == 199:  # 每200个batch打印一次
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(trainloader)}], Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

Epoch [1/10], Batch [200/782], Loss: 1.868
Epoch [1/10], Batch [400/782], Loss: 1.526
Epoch [1/10], Batch [600/782], Loss: 1.368
Epoch [2/10], Batch [200/782], Loss: 1.167
Epoch [2/10], Batch [400/782], Loss: 1.113
Epoch [2/10], Batch [600/782], Loss: 1.059
Epoch [3/10], Batch [200/782], Loss: 0.927
Epoch [3/10], Batch [400/782], Loss: 0.911
Epoch [3/10], Batch [600/782], Loss: 0.905
Epoch [4/10], Batch [200/782], Loss: 0.780
Epoch [4/10], Batch [400/782], Loss: 0.785
Epoch [4/10], Batch [600/782], Loss: 0.769
Epoch [5/10], Batch [200/782], Loss: 0.673
Epoch [5/10], Batch [400/782], Loss: 0.673
Epoch [5/10], Batch [600/782], Loss: 0.689
Epoch [6/10], Batch [200/782], Loss: 0.579
Epoch [6/10], Batch [400/782], Loss: 0.594
Epoch [6/10], Batch [600/782], Loss: 0.599
Epoch [7/10], Batch [200/782], Loss: 0.522
Epoch [7/10], Batch [400/782], Loss: 0.519
Epoch [7/10], Batch [600/782], Loss: 0.542
Epoch [8/10], Batch [200/782], Loss: 0.435
Epoch [8/10], Batch [400/782], Loss: 0.457
Epoch [8/10

In [19]:
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 74.28%
